In [3]:
# **Imports**
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential
from keras import layers
#from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.layers import SimpleRNN, LSTM, GRU
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping


In [39]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

In [40]:
import tensorflow as tf
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping: no known devices.


In [126]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading and Preparing the data

## Loading the Data 

In [8]:
# uploading the files to GC in order to use them. If you use the file in GCP more often, use a DIFFERENT method like connecting to your Drive, like shown below.
#from google.colab import files
#uploaded1 = files.upload()

Saving df_deutschland.csv to df_deutschland.csv


In [9]:
#uploaded2 = files.upload()

Saving wind_hourly_clean.csv to wind_hourly_clean.csv


In [6]:
local_path_target = '/mnt/g/Meine Ablage/raw_data/Target/Hourly/energy_generation_data/df_deutschland.csv'

local_path_feature = "/mnt/g/Meine Ablage/raw_data/Features/DATAFRAMES/wind_hourly_clean.csv"

data_target = pd.read_csv(local_path_target)
data_feature = pd.read_csv(local_path_feature)

#data_target = pd.read_csv('df_deutschland.csv')
#data_feature = pd.read_csv("wind_hourly_clean.csv")

#data_target = pd.read_csv('/content/drive/MyDrive/raw_data/Target/Hourly/energy_generation_data/df_deutschland.csv')
#data_feature = pd.read_csv('/content/drive/MyDrive/raw_data/Features/DATAFRAMES/wind_hourly_clean.csv', index_col=0)

data_target['Date'] = pd.to_datetime(data_target.Date)

In [7]:
generation_df = data_target.groupby(by=data_target['Date']).sum()
generation_df.head(3)

,Wind offshore[MWh],Wind onshore[MWh],Photovoltaics[MWh]
Date,,,
2015-01-01 00:00:00,517.0,8129.0,0.0
2015-01-01 01:00:00,516.0,8300.0,0.0
2015-01-01 02:00:00,514.0,8544.0,0.0


In [8]:
wind_df = generation_df[['Wind offshore[MWh]','Wind onshore[MWh]']].sum(axis=1)
wind_df.head(3)

Date
2015-01-01 00:00:00    8646.0
2015-01-01 01:00:00    8816.0
2015-01-01 02:00:00    9058.0
dtype: float64

In [9]:
data_feature.index = pd.to_datetime(data_feature.index)
data_feature.head(3)

,Date,90,125,161,164,183,197,198,232,282,...,15000,15044,15120,15122,15189,15200,15207,15214,15444,1001
1970-01-01 00:00:00.000000000,2015-01-01 00:00:00,2.1,0.7,0.9,5.1,11.3,3.2,1.6,1.1,0.7,...,3.9,0.8,4.9,3.2,4.4,3.0,0.5,1.2,1.0,2.2
1970-01-01 00:00:00.000000001,2015-01-01 01:00:00,2.7,0.3,0.8,4.9,13.0,3.2,2.6,1.5,0.9,...,3.4,0.7,5.3,2.6,6.0,2.8,0.8,1.1,1.0,2.5
1970-01-01 00:00:00.000000002,2015-01-01 02:00:00,2.5,0.8,1.3,3.9,12.8,3.2,3.2,1.7,0.9,...,3.8,1.3,4.8,2.3,5.7,3.1,0.5,1.8,1.2,2.6


## Preparing the Data




### Concatenating the Data

In [10]:
df_feat_corr = data_feature
df_feat_corr.drop(df_feat_corr.tail(2856).index,inplace = True)

In [11]:
# Building one DataFrame to make the correlation Analysis

complete_df = pd.concat([wind_df, df_feat_corr.set_index(wind_df.index)], axis=1)
complete_df.shape

(52632, 262)

### Filtering the data with highly correlated timeseries

In [12]:
# Correlation Analysis
#complete_df.corr()

In [13]:
df_corr = complete_df.corr().abs()

In [14]:
sorted_corr = df_corr[0].sort_values(ascending=False)
pd.DataFrame(sorted_corr);

In [15]:
list_of_corr_060 = list(sorted_corr[sorted_corr >= 0.6].index)

In [16]:
filtered_df = complete_df[list_of_corr_060]

In [17]:
filtered_df

,0,722,2483,2932,1503,6091,5715,342,1270,963,...,840,1684,7351,1303,368,3623,596,4642,3287,2171
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,8646.0,13.5,7.6,3.4,3.3,6.9,4.1,3.9,4.0,3.3,...,1.4,2.9,5.5,3.2,1.4,0.6,8.8,4.7,2.1,2.0
2015-01-01 01:00:00,8816.0,14.7,6.7,3.1,4.0,6.9,4.0,3.2,4.1,3.6,...,1.1,3.3,6.1,3.3,1.7,0.7,7.8,4.9,2.1,1.9
2015-01-01 02:00:00,9058.0,13.6,6.9,4.0,4.4,6.9,4.8,3.4,4.3,4.6,...,1.4,2.7,5.5,3.0,1.8,1.2,7.1,4.9,2.1,2.1
2015-01-01 03:00:00,9071.0,13.0,7.2,4.3,4.7,6.9,5.0,3.5,3.7,3.8,...,1.3,2.8,5.8,3.0,1.9,1.1,6.6,6.1,1.8,2.1
2015-01-01 04:00:00,9168.0,14.4,7.9,4.0,4.8,6.9,3.8,2.6,2.7,4.3,...,1.8,3.1,5.3,2.6,1.9,1.5,7.1,5.2,1.6,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-01 19:00:00,4845.0,9.2,3.3,1.7,3.2,5.2,3.7,2.1,2.1,3.3,...,1.3,2.4,3.0,2.3,1.9,1.3,5.0,3.2,0.9,0.6
2021-01-01 20:00:00,5028.0,9.9,3.5,2.0,2.9,5.3,4.1,1.8,1.7,3.6,...,0.8,3.4,2.7,1.8,1.8,0.2,5.0,3.0,0.7,0.9
2021-01-01 21:00:00,5204.0,9.7,4.1,2.2,3.0,6.2,3.8,2.5,1.6,3.4,...,0.4,2.8,2.0,2.4,1.8,1.4,5.0,3.1,1.1,1.1


### Scaling the Data

In [18]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(filtered_df)

In [19]:
np.asmatrix(scaled)

matrix([[0.11213717, 0.34300792, 0.34123223, ..., 0.27777778, 0.15      ,
         0.12987013],
        [0.11434204, 0.37467018, 0.2985782 , ..., 0.29012346, 0.15      ,
         0.12337662],
        [0.11748074, 0.34564644, 0.30805687, ..., 0.29012346, 0.15      ,
         0.13636364],
        ...,
        [0.06749501, 0.24274406, 0.17535545, ..., 0.17901235, 0.07857143,
         0.07142857],
        [0.07368162, 0.25593668, 0.16587678, ..., 0.22839506, 0.09285714,
         0.06493506],
        [0.06916811, 0.25329815, 0.17535545, ..., 0.22839506, 0.05714286,
         0.02597403]])

In [20]:
scaled[0].shape

(123,)

In [21]:
len(scaled)*0.8

42105.600000000006

## Forming the Sequences for RNN

In [22]:
# Train_Data
trainX = scaled[:int(len(scaled)*0.8),1:]
trainy = scaled[0:int(len(scaled)*0.8),0]
# Test Data
testX = scaled[int(len(scaled)*0.8):len(scaled)-366,1:]
testy = scaled[int(len(scaled)*0.8):len(scaled)-366,0]
# Validation Data
x_val = scaled[len(scaled)-366:,1:]
y_val = scaled[len(scaled)-366:,0]

In [23]:
testX.shape

(10161, 122)

In [24]:
timesteps = 8760
train_generator = TimeseriesGenerator(trainX, trainy, length=timesteps, sampling_rate=1, batch_size=64)
validation_generator = TimeseriesGenerator(testX, testy, length=timesteps, sampling_rate=1, batch_size=64)
test_generator = TimeseriesGenerator(x_val, y_val, length=336, sampling_rate=1, batch_size=64)

In [25]:
X_train, y_train = train_generator[0]
X_test, y_test = validation_generator[0]

train_samples = X_train.shape[0]*len(train_generator)
test_samples = X_test.shape[0]*len(test_generator)

In [26]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((64, 8760, 122), (64,), (64, 8760, 122), (64,))

In [27]:
(122,8760,1) , (8760,1)

((122, 8760, 1), (8760, 1))

In [28]:
y_train

array([0.11218905, 0.11128116, 0.11081425, 0.1095432 , 0.10762367,
       0.09495214, 0.08526368, 0.07295531, 0.06027081, 0.04844232,
       0.03605613, 0.02376073, 0.01671811, 0.01025914, 0.00644601,
       0.00526575, 0.00866385, 0.01684781, 0.02787217, 0.04378615,
       0.05684677, 0.07019273, 0.08020544, 0.09030894, 0.0968198 ,
       0.11370652, 0.13901066, 0.15501543, 0.17450909, 0.18989131,
       0.20079894, 0.20837332, 0.21516952, 0.22472828, 0.23204327,
       0.23694586, 0.23777593, 0.23855412, 0.24047366, 0.24330108,
       0.24894296, 0.25830718, 0.2622111 , 0.26079739, 0.25545381,
       0.24907266, 0.2394231 , 0.24349563, 0.24546704, 0.24123888,
       0.24127779, 0.23917668, 0.23715338, 0.23739981, 0.23894322,
       0.23593422, 0.23025343, 0.22312002, 0.21641462, 0.20275739,
       0.18899639, 0.18627273, 0.18568909, 0.19303001])

# The Model
The PlanA from Research AND before Testing
Long Short Term Memory (LSTM) neural networks

3 main elements:
- Deseasonalization
- adaptive normalization:
- Stochastic Gradient Descent (SGD)

## Model_1 , LSTM Model

In [29]:
#Do I need to normalize the Data?

# First Model guess
# input_dim=1 because we have only wind speed feature
# RELU because provides only values above 0, we cannot get - generation data - Try Tanh and Sigmoid too in later tries
# **Last Layer** layers.Dense(336) because we have 336 hours in the 2 week prediction. Or do I put 1 because I have 1 X for every output. 
# 
# "linear" According to LEcture has to be like that for regression
# COMPILE LAYER for regression problem: 
# loss='mse', 
# optimizer='adam', (to start with) Choose SGD AND RMSProp(good for RNN) for further testing
# metrics=['mae', "mse"]) (we can use muliple metrics in order to compare)

# 2 units because the time aspect of the series does not have much prediction value. Better to emphasize feature data.

# Try stacked Layers 1,2,3,4

model_1 = Sequential()
model_1.add(LSTM(128,input_shape=(8760,122),return_sequences=True ))
model_1.add(layers.LeakyReLU(alpha=0.5))
#model_1.add(LSTM(128, return_sequences=True))
#model_1.add(layers.LeakyReLU(alpha=0.5))
model_1.add(layers.Dropout(0.3))
model_1.add(LSTM(128, return_sequences=False))
model_1.add(layers.Dropout(0.3))
model_1.add(layers.Dense(1))


In [30]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 8760, 128)         128512    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 8760, 128)         0         
_________________________________________________________________
dropout (Dropout)            (None, 8760, 128)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 260,225
Trainable params: 260,225
Non-trainable params: 0
__________________________________________________

In [41]:
es = [EarlyStopping(monitor="val_loss", mode='min', patience = 10, restore_best_weights=True)] 

model_1.compile(loss='mse', optimizer='adam', metrics=['mae', "mse"])

history = model_1.fit(train_generator,validation_data=validation_generator, shuffle = False,  callbacks = es, epochs=1, verbose=2)


KeyboardInterrupt: 

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"], color="red")

In [ ]:
testX = np.expand_dims(scaled[len(scaled)-336:,1:], axis=0)

In [ ]:
evaluation = model_1.evaluate(X_test,y_test,verbose=0)

In [ ]:
testX.shape

In [ ]:
result = model_1.predict_generator(test_generator)

In [ ]:
result.shape

In [ ]:
testy = scaled[len(scaled)-336:,0]

In [ ]:
plt.plot(result.flatten())
plt.plot(testy, color="red")

### Saving the Model

In [ ]:
checkpoint = {'input_size': 784,
              'output_size': 10,
              'hidden_layers': [each.out_features for each in model.hidden_layers],
              'state_dict': model.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

In [ ]:
def save_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = fc_model.Network(checkpoint['input_size'],
                             checkpoint['output_size'],
                             checkpoint['hidden_layers'])
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

### Testing the MAPE score

In [ ]:
def mae(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sum(np.abs(y_true-y_pred))/(len(y_true))

def mse(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.mean((y_true-y_pred)**2)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
testy = pd.DataFrame(testy)
testy[0];

In [ ]:
res = pd.DataFrame(result.flatten())
res[0];

In [ ]:
mean_absolute_percentage_error(testy[0], res[0])

Ideas to improve the model:
Does the split into 64 sequences make sense? Try 32?
try out different stride size in sequencing, maybe even tumbling 
window and no overlapping window

Multiple Layers:


### Unscaling and Comparing